In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import warnings

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [3]:
def prise_de_glace(df, alert, pg, window_size, i):
    
    df["var_S03"] = df["S03"] - df["S03"].shift(1)
    df["var_S04"] = df["S04"] - df["S04"].shift(1)
    
    def rolling_mediane(serie, window):
        valeur = [serie.iloc[0]]
        for i in range(1,len(serie)):
            tableau = serie[max(0, i-window+1):i+1]
            valeur.append(np.median(tableau))
        return valeur

    def rolling_moyenne(serie, window):
        valeur = [serie.iloc[0]]
        for i in range(1,len(serie)):
            tableau = serie[max(0, i-window+1):i+1]
            valeur.append(np.mean(tableau))
        return valeur
    
    df["difference"] = df["S03"] - df["S04"]
    df["difference_future"] = rolling_moyenne(df["difference"], window_size)
    df["var_S03_future"] = rolling_moyenne(df["var_S03"], window_size)
    df["var_S03_future_expo"] = np.exp(df["var_S03_future"])
    df["var_S04_future"] = rolling_moyenne(df["var_S04"], window_size)
    df["var_S04_future_expo"] = np.exp(df["var_S04_future"])
    df["taux_relatif"] = df["var_S03_future_expo"] / df["var_S04_future_expo"]
    df["taux_relatif_future"] = rolling_moyenne(df["taux_relatif"], window_size)
    df["taux_relatif_difference_future"] = df["taux_relatif_future"]*df["difference_future"]
    
    if i < window_size:
        alert = False
        df.iloc[-1, df.columns.get_loc('normale?')] = 1
        return alert, df, pg
    else:
        normal_frame = df[df["normale?"] == 1]
        
        start_index = 0
        df1 = normal_frame.iloc[start_index:]
        mean_median_rolling = df1["taux_relatif_difference_future"].mean()
        std_median_rolling = df1["taux_relatif_difference_future"].std()
        n = df1["taux_relatif_difference_future"].count() 
        confidence_level = 0.99 # pourcentage d'erreur toléré
        z_score = stats.norm.ppf((1 + confidence_level) / 2)
        confidence_interval_lower = mean_median_rolling - z_score * std_median_rolling
        confidence_interval_upper = mean_median_rolling + z_score * std_median_rolling
        
        if(df.iloc[-1, df.columns.get_loc('taux_relatif_difference_future')] < confidence_interval_lower or 
        df.iloc[-1, df.columns.get_loc('taux_relatif_difference_future')] > confidence_interval_upper):
            if(df.iloc[-2, df.columns.get_loc('normale?')] == 1):
                alert = True
        else:
            alert = False
            df.iloc[-1, df.columns.get_loc('normale?')] = 1 
            if(df.iloc[-2, df.columns.get_loc('normale?')] == 0):
                print("fin de prise de glace le ", df.index[-1])
                pg.iloc[-1, pg.columns.get_loc('fin')] = df.index[-1]
        
        return alert, df, pg

In [4]:
file_path = 'dataset/Lec_Luneville_Mai2023-2024_24.csv'
df = pd.read_csv(file_path, sep=';')
df = df.set_index('date')
df.index = pd.to_datetime(df.index, format='%d/%m/%Y %H:%M', errors='coerce')
df.sort_values(by=['date'], inplace=True)
df['S03'] = df['S03'].str.replace(',', '.')
df['S04'] = df['S04'].str.replace(',', '.')
df['S03'] = pd.to_numeric(df['S03'], errors='coerce')
df['S04'] = pd.to_numeric(df['S04'], errors='coerce')
df['S03'].fillna(df['S03'].median(), inplace=True)
df['S04'].fillna(df['S04'].median(), inplace=True)

In [5]:
alert = False
window_size = 288
serie5 = pd.DataFrame(columns=df.columns)
pg5 = pd.DataFrame(columns=["numero", "debut", "fin"])
serie5['normale?'] = None
serie5 = pd.concat([serie5, df.iloc[[1]]], ignore_index=False)
serie5.loc[df.index[1], 'normale?'] = 1
j = 0
cumul_alert = False
serie5.to_csv('serie5.csv', index=True)
pg5.to_csv('pg5.csv', index=True)
for i in range(2, len(df)):
    if alert:
        j += 1
        print("La ", j, "ième prise de glace a été détectée à la date ", serie5.index[-1])
        new_row = pd.DataFrame([[j, serie5.index[-1], serie5.index[-1]]], columns=["numero", "debut", "fin"])
        pg5 = pd.concat([pg5, new_row], ignore_index=True)
    serie5 = pd.concat([serie5, df.iloc[[i]]], ignore_index=False)
    serie5.loc[df.index[i], 'normale?'] = 0
    alert = False
    alert, serie5, pg5 = prise_de_glace(serie5, alert, pg5, window_size, i)
    #serie5.to_csv('serie5.csv', index=True)
    cumul_alert = cumul_alert or alert
pg5.to_csv('pg5.csv', index=True)

if not cumul_alert:
    print("Pas de prise de glace détectée")
else:
    print("\nFin de la détection de prise de glace")


La  1 ième prise de glace a été détectée à la date  2023-07-02 14:25:00
fin de prise de glace le  2023-07-03 12:40:00
La  2 ième prise de glace a été détectée à la date  2023-07-05 22:55:00
fin de prise de glace le  2023-07-06 14:20:00
La  3 ième prise de glace a été détectée à la date  2023-07-06 14:30:00
fin de prise de glace le  2023-07-06 14:35:00
La  4 ième prise de glace a été détectée à la date  2023-09-08 03:15:00
fin de prise de glace le  2023-09-08 09:20:00
La  5 ième prise de glace a été détectée à la date  2023-09-24 14:25:00
fin de prise de glace le  2023-09-25 11:35:00
